In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os

In [2]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['SHG_Real_Almirror__1_0Polaz_07312017.txt', 'SHG_Real_Almirror__1_90Polaz_07312017.txt']


In [3]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [4]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 0, skipfooter = 0, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)


In [5]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (615*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay = []
    for i in range(0,len(x)):
        timedelay = -((x[i,0]*itr + offset)*25.4*2/(10**-12*c))+T1
        timeDelay.append(timedelay)
    return(timeDelay)

In [6]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,900):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
    
times = times.values.reshape(900,len(allFiles))
PDvalues = PDvalues.values.reshape(900,len(allFiles)) 
timeDelay = ConvertToTimeDelay(times,itr,offset) # function converts to timedelay
P_Polar = PDvalues[:,0]
S_Polar = PDvalues[:,1]

Baseline = np.mean(P_Polar[100:300]) # Getting average baseline value before Time Zero
Enhancement = P_Polar[100:900].min(axis = 0)/Baseline
print(Enhancement)

1.39253825335


In [8]:
plt.plot(timeDelay,P_Polar,'k',label = 'P-pol')
plt.plot(timeDelay,S_Polar,'r',label = 'S-pol')
extraticks = [-10,-9,-8,-7,-6,-5,-4, -3, -2, -1, 1, 2, 3, 4,5,6,7,8,9,10]
plt.xlabel('Delay(ps)')
plt.ylabel('PMT Signal')
plt.grid(True)
plt.xticks(list(plt.xticks()[0]) + extraticks)
plt.xlim(-11,11)
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
print(P_Polar.argmin(axis = 0))

In [ ]:
print(np.mean(P_Polar[100:300]))